In [1]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00


In [2]:
import gradio as gr
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
from diffusers import StableDiffusionInpaintPipeline

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Stable Diffusion Inpainting Model
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting"
).to(device)

print("✅ Model Loaded Successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Model Loaded Successfully!


In [9]:
import numpy as np
import dlib
import cv2
from PIL import Image
import torch
import gradio as gr
import os
import urllib.request
from diffusers import StableDiffusionInpaintPipeline

# Load Inpainting Model
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting"
).to(device)

# Load Dlib Front-Face Landmarks
predictor_path = "/content/drive/My Drive/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Download OpenCV RetinaFace-like model
proto_url = "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt"
model_url = "https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel"
proto_path = "deploy.prototxt"
model_path = "res10_300x300_ssd_iter_140000.caffemodel"

if not os.path.isfile(proto_path):
    urllib.request.urlretrieve(proto_url, proto_path)
if not os.path.isfile(model_path):
    urllib.request.urlretrieve(model_url, model_path)

face_net = cv2.dnn.readNetFromCaffe(proto_path, model_path)

# Nose style prompts
front_prompts = {
    "Small": "Front view portrait of a person with a small, delicate nose...",
    "Sharp": "Front view portrait of a person with a sharp, well-defined nose...",
    "Round": "Front view portrait of a person with a round, bulbous nose...",
    "Wide": "Front view portrait of a person with a wide, flat nose...",
    "Natural": "Front view portrait of a person with a natural nose...",
    "Pointed": "Front view portrait of a person with a pointed, slender nose...",
    "Button": "Front view portrait of a person with a cute button nose...",
    "Greek": "Front view portrait of a person with a straight Greek nose...",
    "Roman": "Front view portrait of a person with a Roman nose with a high bridge...",
    "Snub": "Front view portrait of a person with a snub, upturned nose..."
}

side_prompts = {
    "Small": "Side view portrait of a woman with a small, delicate nose...",
    "Sharp": "Side profile of a woman with a sharp, defined nose...",
    "Round": "Side view portrait of a woman with a round, bulbous nose...",
    "Wide": "Side profile of a woman with a wide, broad nose...",
    "Natural": "Side view portrait of a woman with a naturally shaped nose...",
    "Pointed": "Side profile of a woman with a pointed, slender nose...",
    "Greek": "Side view of a woman with a straight Greek nose...",
    "Roman": "Side view portrait of a woman with a Roman nose, strong bridge...",
    "Snub": "Side view portrait of a woman with a snub, upturned nose...",
    "Button": "Side profile of a woman with a cute button nose..."
}

# Improved RetinaFace-like detection for side view
def detect_side_face_retina(image):
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
                                 (104.0, 177.0, 123.0))
    face_net.setInput(blob)
    detections = face_net.forward()
    h, w = image.shape[:2]
    max_confidence = 0
    face_box = None

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > max_confidence and confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            face_box = box.astype("int")
            max_confidence = confidence

    return face_box

# Nose region estimation
def estimate_nose_region(img_bgr, view_type):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    h, w = img_bgr.shape[:2]

    if view_type == "Front View":
        faces = detector(gray)
        if len(faces) == 0:
            return None

        landmarks = predictor(gray, faces[0])
        nose_points = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(27, 36)]
        x_coords = [pt[0] for pt in nose_points]
        y_coords = [pt[1] for pt in nose_points]

        nose_x = min(x_coords)
        nose_y = min(y_coords)
        nose_width = max(x_coords) - nose_x
        nose_height = max(y_coords) - nose_y
        return nose_x, nose_y, nose_width, nose_height

    else:
        face_box = detect_side_face_retina(img_bgr)
        if face_box is None:
            return None

        x1, y1, x2, y2 = face_box
        face_w = x2 - x1
        face_h = y2 - y1

        # Adjusted for better generalization to side images
        nose_x = x1 + int(face_w * 0.3)
        nose_y = y1 + int(face_h * 0.45)
        nose_width = int(face_w * 0.2)
        nose_height = int(face_h * 0.15)
        return nose_x, nose_y, nose_width, nose_height

# Inpainting process
def modify_nose_view(input_image, view_type, nose_style):
    input_image = input_image.resize((512, 512))
    img_array = np.array(input_image)
    img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

    nose_box = estimate_nose_region(img_bgr, view_type)
    if nose_box is None:
        return "❌ No face/nose detected. Please try a clearer or better-angled side image."

    nose_x, nose_y, nose_width, nose_height = nose_box
    prompt = (front_prompts if view_type == "Front View" else side_prompts).get(nose_style, "natural nose")

    # Mask
    mask = np.zeros((512, 512), dtype=np.uint8)
    center = (nose_x + nose_width // 2, nose_y + nose_height // 2)
    axes = (nose_width, nose_height)
    cv2.ellipse(mask, center, axes, 0, 0, 360, 255, -1)
    mask = cv2.GaussianBlur(mask, (15, 15), 0)
    mask_image = Image.fromarray(mask).convert("L")

    # Inpainting
    result = pipe(
        prompt=prompt,
        negative_prompt="blurry, distorted, artifact, bad nose",
        image=input_image,
        mask_image=mask_image,
        guidance_scale=7.5,
        num_inference_steps=50,
        strength=0.75
    )

    return result.images[0]

# Update available nose styles
def update_nose_styles(view_type):
    return gr.update(choices=list(front_prompts if view_type == "Front View" else side_prompts), value="Natural")

# Gradio Interface
with gr.Blocks() as iface:
    gr.Markdown("# 🧠 Rhinoplasty AI")
    gr.Markdown("Upload a **front-view** or **side-view** face image and choose a nose style to modify.")

    with gr.Row():
        img_input = gr.Image(type="pil", label="Upload Face Image")
        view_input = gr.Radio(["Front View", "Side View"], label="Select View", value="Front View")
        style_input = gr.Radio(list(front_prompts), label="Select Nose Style")

    output_img = gr.Image(label="Modified Output")

    view_input.change(fn=update_nose_styles, inputs=view_input, outputs=style_input)
    gr.Button("Modify Nose").click(fn=modify_nose_view, inputs=[img_input, view_input, style_input], outputs=output_img)

iface.launch(share=True)
# lalala

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e8f5cd78faf02b1dc9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
